In [ ]:
## Importing the libraries
import numpy as np
import pandas as pd
import keras
from keras import backend as k
from keras.models import Sequential
from keras.layers import Activation
from keras.layers.core import Dense , Flatten
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import *
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import itertools
import random
import os
import cv2
import pickle
from keras.utils import to_categorical

from sklearn.model_selection import train_test_split

In [ ]:
## Defining the folder to import the images from main kaggle directory
main_path = "../input/severstal-steel-defect-detection/"
sub_folder = ["train_images"]


In [ ]:
IMAGE_SIZE=100

name =[]
image =[]

for i in sub_folder:
    folder = os.path.join(main_path,i)
    label = sub_folder.index(i)
    for j in os.listdir(folder):
        path = os.path.join(folder,j)
        img_arr = cv2.imread(path)
        img_arr = cv2.resize(img_arr , (IMAGE_SIZE,IMAGE_SIZE))
        image.append(img_arr)
        name.append(str(j))


In [ ]:
df = pd.DataFrame(data={"ImageId":name,"img":image})


In [ ]:
df.shape[0]

In [ ]:
from matplotlib import pyplot as plt
plt.figure(figsize=[5,5])

plt.subplot(121)
plt.imshow(df["img"][0], cmap='gray')
plt.title("Image Name : {}".format(df["ImageId"][0]))


In [ ]:
data= pd.read_csv("../input/severstal-steel-defect-detection/train.csv")

In [ ]:
data.head()

In [ ]:
data["ClassId"].value_counts()

In [ ]:
table=pd.merge(df, data, on='ImageId')

In [ ]:
Sample = []
for i in table["img"]:
    Sample.append(i)
    


In [ ]:
Label = table["ClassId"].values

In [ ]:
Sample = np.asarray(Sample).astype('float32')
Label = np.asarray(Label).astype('float32')

In [ ]:
trainSample = Sample[:-1419]
testSample = Sample[:1419]
trainLabel = Label[:-1419]
testLabel =Label[:1419]

In [ ]:
print(trainSample.shape)
print(testSample.shape)
print(trainLabel.shape)
print(testLabel.shape)

In [ ]:
Sample[0].shape

In [ ]:
X_train, X_test , y_train , y_test = train_test_split(trainSample, trainLabel,test_size=0.33 , random_state=42)

In [ ]:
model = Sequential([
    Conv2D(32,(3,3),activation ='relu',input_shape=(100,100,3)),
    Flatten(),
    Dense(5,activation='softmax')
])

In [ ]:
model.compile(Adam(lr=.0001),loss='sparse_categorical_crossentropy',metrics=['accuracy'])


In [ ]:
model.summary()

In [ ]:
batch_size = 64
epochs = 20
num_classes = 10


In [ ]:
model_ima = model.fit(X_train,y_train, batch_size=batch_size,epochs=epochs,verbose=2,validation_data=(X_test, y_test))


In [ ]:
prediction =model.predict_classes(testSample)

In [ ]:
cm =confusion_matrix(testLabel,prediction)

In [ ]:
cm

In [ ]:

def plot_confusion_matrix(cm,
                          target_names,
                          title='Confusion matrix',
                          cmap=None,
                          normalize=True):
    """
    given a sklearn confusion matrix (cm), make a nice plot

    Arguments
    ---------
    cm:           confusion matrix from sklearn.metrics.confusion_matrix

    target_names: given classification classes such as [0, 1, 2]
                  the class names, for example: ['high', 'medium', 'low']

    title:        the text to display at the top of the matrix

    cmap:         the gradient of the values displayed from matplotlib.pyplot.cm
                  see http://matplotlib.org/examples/color/colormaps_reference.html
                  plt.get_cmap('jet') or plt.cm.Blues

    normalize:    If False, plot the raw numbers
                  If True, plot the proportions

    Usage
    -----
    plot_confusion_matrix(cm           = cm,                  # confusion matrix created by
                                                              # sklearn.metrics.confusion_matrix
                          normalize    = True,                # show proportions
                          target_names = y_labels_vals,       # list of names of the classes
                          title        = best_estimator_name) # title of graph

    Citiation
    ---------
    http://scikit-learn.org/stable/auto_examples/model_selection/plot_confusion_matrix.html

    """
    import matplotlib.pyplot as plt
    import numpy as np
    import itertools

    accuracy = np.trace(cm) / float(np.sum(cm))
    misclass = 1 - accuracy

    if cmap is None:
        cmap = plt.get_cmap('Blues')

    plt.figure(figsize=(8, 6))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()

    if target_names is not None:
        tick_marks = np.arange(len(target_names))
        plt.xticks(tick_marks, target_names, rotation=45)
        plt.yticks(tick_marks, target_names)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]


    thresh = cm.max() / 1.5 if normalize else cm.max() / 2
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        if normalize:
            plt.text(j, i, "{:0.4f}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")
        else:
            plt.text(j, i, "{:,}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")


    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label\naccuracy={:0.4f}; misclass={:0.4f}'.format(accuracy, misclass))
    plt.show()

In [ ]:
cmplot=[1,2,3,4]
plot_confusion_matrix(cm,cmplot)